<a href="https://colab.research.google.com/github/CalculatedContent/WeightWatcher/blob/master/examples/WW_Calibrate_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Calibrate  Alpha against Random Distance metric

This notebook is an attempt to calibrate the PL fits of $\alpha$ to the Rand-Distance metric, using a Pareto random matrix (as opposed to a strongly correlated matrix)

Notice that the Rand_Distance metrix depends on Q

see also:  https://github.com/CalculatedContent/ImplicitSelfRegularization/blob/master/HeavyTailedUniversalityClasses.ipynb



In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm


In [ ]:
!pip install weightwatcher

In [2]:
import weightwatcher as ww
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(ww.__NAME__) 
logger.setLevel(logging.INFO)


### Create dummy pyTorch model

see:  https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html


In [3]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self, W):
        super(TinyModel, self).__init__()
        N, M = W.shape[0], W.shape[1]
        assert(N >= M)
        self.linear = torch.nn.Linear(N, M, bias=False)
        self.linear.weight.data = torch.from_numpy(W)
        
    def forward(self, x):
        x = self.linear(x)
        return x


In [ ]:
#%%capture
from IPython.utils import io

import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger('weightwatcher') 
logger.setLevel(logging.CRITICAL)


all_details = pd.DataFrame()

for mu in tqdm([2, 2.5, 3, 3.5, 4, 4.5]):
    for M in [100, 250, 500, 1000, 2000]:
        for Q in [1,2,3,4,5,6,7,8,8,9,10]:
            N = Q*M
            W = np.random.pareto(mu,size=(N,M))
            model = TinyModel(W)

            #print(f"mu={mu} M={M} Q={Q}")
            # supress print statements from powerlaw fits
            with io.capture_output() as captured:
                watcher = ww.WeightWatcher(model=model)
                details = watcher.analyze(mp_fit=True, randomize=True)
                details['mu']=mu
                details['Q']=Q
                all_details = pd.concat([all_details, details])

 67%|██████▋   | 4/6 [46:18<23:46, 713.24s/it]

In [ ]:
all_details.plot.scatter(x="Q", y='alpha')

In [ ]:
all_details.plot.scatter(x="rand_distance", y='alpha')

In [ ]:
all_details.plot.scatter(x="rand_distance", y='Q')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

for Q in [1,3,5,7,9]:
    alpha = all_details[all_details.Q==1].alpha.to_numpy()
    rand_distance = all_details[all_details.Q==Q].rand_distance.to_numpy()
    idx = np.argsort(rand_distance)
    if len(alpha)==len(rand_distance):
        plt.scatter(rand_distance[idx], alpha[idx], label=str(Q))
plt.legend()
plt.show()
